In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
file = r"C:\Users\HP\OneDrive\Desktop\Arit data analysis\WEEK1 CHALLENGE\cleaned_data.csv"
data = pd.read_csv(file)
data

In [3]:
data["InvoiceDate"] = pd.to_datetime(data["InvoiceDate"], errors="coerce")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  int64         
 1   StockCode    541909 non-null  int64         
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   541909 non-null  int64         
 7   Country      541909 non-null  object        
 8   Totalprice   541909 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(2)
memory usage: 37.2+ MB


In [4]:
#Are there Descriptions generating very little revenue despite being in stock?
summary = len(data[data["Totalprice"] < 100])
summary

527754

In [5]:
#Which Descriptions have the highest quantity sold overall?
description = data.loc[data["Quantity"] == data["Quantity"].max() , ["Description"]]
description.to_csv("maxquantity.csv")

In [ ]:
def bestseller(data):
    data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
    data['YearMonth'] = data['InvoiceDate'].dt.to_period('M')  

    monthly_sales = data.groupby(['Description', 'YearMonth'])['Quantity'].sum().reset_index()
    months_sold = monthly_sales[monthly_sales['Quantity'] > 0].groupby('Description').count()['YearMonth']

    total_months = data['YearMonth'].nunique()
    consistent_sellers = months_sold[months_sold == total_months]

    print("Consistent sellers across all months:")
    consistent_sellers.to_csv("bestsellersmonths.csv")
bestseller(data)

Consistent sellers across all months:


In [10]:
#Which Descriptions have very low sales quantities?
value = len(data.loc[(data["Quantity"] <= 1000) & (data['Quantity'] != 0), ['Description']])
value

541735

In [ ]:
#productss not purchased
not_purchased = data.loc[(data['UnitPrice'] == 0.0) & (data["Quantity"]== 0.0), ['Description']]
not_purchased # all purchased

,Description


In [ ]:
# Most frequently purchased product
frequently = data["Description"].mode()[0]
print(frequently)


WHITE HANGING HEART T-LIGHT HOLDER


In [ ]:
store_products = data.groupby(["Country", "Description"]).agg({
    'Quantity': 'sum',
    'Totalprice': 'sum'
}).reset_index()
top_per_country = store_products.loc[
    store_products.groupby("Country")["Totalprice"].idxmax()
]
top_per_country.to_csv("Maxproductspercountry.csv")


In [60]:
#Which products are candidates for discount promotions (low revenue but medium demand
discount = data.loc[(data['Totalprice']< 5000 )& (data["Quantity"].isin(range(5000,30000))) ]
discount.to_csv("productsdiscount.csv")

In [ ]:


data = data[data["Description"].apply(lambda x: isinstance(x, str))].copy()
data["Description"] = data["Description"].str.lower()

vectorizer = CountVectorizer(stop_words="english", ngram_range=(2,2))
X = vectorizer.fit_transform(data["Description"]) 

totalprice = data["Totalprice"].values
revenue_contrib = X.T @ totalprice

phrase_revenue = pd.DataFrame({
    "Phrase": vectorizer.get_feature_names_out(),
    "TotalRevenue": revenue_contrib.ravel() 
}).sort_values("TotalRevenue", ascending=False)
phrase_revenue.to_csv("bigrams_revenue.csv", index=False)


In [ ]:
#frequently bought year
group_year = data.groupby(data["InvoiceDate"].dt.month_name())["Description"] \
                 .agg(lambda x: x.mode()[0])  # mode per group
group_year.to_csv("month.csv")